"A. Jain, “Automatic Extractive Text Summarization using TF-IDF,” 01-Apr-2019. [Online]. Available: https://medium.com/voice-tech-podcast/automatic-extractive-text-summarization-using-tfidf-3fc9a7b26f5. [Accessed: 24-Nov-2020]."

https://pypi.org/project/rouge/

Cosine similarity : https://www.geeksforgeeks.org/python-measure-similarity-between-two-sentences-using-cosine-similarity/



In [4]:
import nltk
import os
import re
import math
import csv
import operator
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize,word_tokenize
import pandas as pd
nltk.download('averaged_perceptron_tagger')
Stopwords = set(stopwords.words('english'))
wordlemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\risha\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [5]:
def lemmatize_words(words):
    lemmatized_words = []
    for word in words:
        lemmatized_words.append(wordlemmatizer.lemmatize(word))
    return lemmatized_words
def stem_words(words):
    stemmed_words = []
    for word in words:
       stemmed_words.append(stemmer.stem(word))
    return stemmed_words
def remove_special_characters(text):
    regex = r'[^a-zA-Z0-9\s]'
    text = re.sub(regex,'',text)
    return text
def freq(words):
    words = [word.lower() for word in words]
    dict_freq = {}
    words_unique = []
    for word in words:
       if word not in words_unique:
           words_unique.append(word)
    for word in words_unique:
       dict_freq[word] = words.count(word)
    return dict_freq
def pos_tagging(text):
    pos_tag = nltk.pos_tag(text.split())
    pos_tagged_noun_verb = []
    for word,tag in pos_tag:
        if tag == "NN" or tag == "NNP" or tag == "NNS" or tag == "VB" or tag == "VBD" or tag == "VBG" or tag == "VBN" or tag == "VBP" or tag == "VBZ":
             pos_tagged_noun_verb.append(word)
    return pos_tagged_noun_verb
def tf_score(word,sentence):
    freq_sum = 0
    word_frequency_in_sentence = 0
    len_sentence = len(sentence)
    for word_in_sentence in sentence.split():
        if word == word_in_sentence:
            word_frequency_in_sentence = word_frequency_in_sentence + 1
    tf =  word_frequency_in_sentence/ len_sentence
    return tf
def idf_score(no_of_sentences,word,sentences):
    no_of_sentence_containing_word = 0
    for sentence in sentences:
        sentence = remove_special_characters(str(sentence))
        sentence = re.sub(r'\d+', '', sentence)
        sentence = sentence.split()
        sentence = [word for word in sentence if word.lower() not in Stopwords and len(word)>1]
        sentence = [word.lower() for word in sentence]
        sentence = [wordlemmatizer.lemmatize(word) for word in sentence]
        if word in sentence:
            no_of_sentence_containing_word = no_of_sentence_containing_word + 1
    idf = math.log10(no_of_sentences/no_of_sentence_containing_word)
    return idf
def tf_idf_score(tf,idf):
    return tf*idf
def word_tfidf(dict_freq,word,sentences,sentence):
    word_tfidf = []
    tf = tf_score(word,sentence)
    idf = idf_score(len(sentences),word,sentences)
    tf_idf = tf_idf_score(tf,idf)
    return tf_idf
def sentence_importance(sentence,dict_freq,sentences):
    sentence_score = 0
    sentence = remove_special_characters(str(sentence)) 
    sentence = re.sub(r'\d+', '', sentence)
    pos_tagged_sentence = [] 
    no_of_sentences = len(sentences)
    pos_tagged_sentence = pos_tagging(sentence)
    for word in pos_tagged_sentence:
        if word.lower() not in Stopwords and word not in Stopwords and len(word)>1: 
            word = word.lower()
            word = wordlemmatizer.lemmatize(word)
            sentence_score = sentence_score + word_tfidf(dict_freq,word,sentences,sentence)
    return sentence_score

In [6]:
df = pd.read_csv("test.csv")
df

,Unnamed: 0,Article,Summary
0,572,French film A Very Long Engagement has receive...,Last November a court judged the film was too ...
1,2067,"If you have finished Doom 3, Half Life 2 and H...",Released 1 March on PC.Released 11 February on...
2,1566,Manchester City striker Nicolas Anelka has iss...,No-one has come to me and said I would like to...
3,644,The US Vibe awards will be held again next yea...,Rapper Young Buck has been charged after alleg...
4,217,South Africa biggest retail bank Absa has said...,South Africa biggest retail bank Absa has said...
...,...,...,...
85,537,Harry Potter director Alfonso Cuaron has signe...,His first film for Warner Brothers was 1995's ...
86,713,US comedian Bill Cosby will not face charges s...,"Cosby's lawyer, Walter M Phillips Jr, said the..."
87,1370,Jason Gardener fought all the way to the line ...,also set a new championship mark in the men's ...
88,584,Mike Leigh's award-winning abortion drama Vera...,Considine is also nominated for a best support...


In [7]:
para = ""
for id,row in df.iterrows():
    text = row['Article']
    tokenized_sentence = sent_tokenize(text)
    text = remove_special_characters(str(text))
    text = re.sub(r'\d+', '', text)
    tokenized_words_with_stopwords = word_tokenize(text)
    tokenized_words = [word for word in tokenized_words_with_stopwords if word not in Stopwords]
    tokenized_words = [word for word in tokenized_words if len(word) > 1]
    tokenized_words = [word.lower() for word in tokenized_words]
    tokenized_words = lemmatize_words(tokenized_words)
    word_freq = freq(tokenized_words)
#     input_user = int(input('Percentage of information to retain(in percent):'))
    input_user = 50
    
    no_of_sentences = int((input_user * len(tokenized_sentence))/100)
    print(no_of_sentences)
    c = 1
    sentence_with_importance = {}
    for sent in tokenized_sentence:
        sentenceimp = sentence_importance(sent,word_freq,tokenized_sentence)
        sentence_with_importance[c] = sentenceimp
        c = c+1
    sentence_with_importance = sorted(sentence_with_importance.items(), key=operator.itemgetter(1),reverse=True)
    cnt = 0
    summary = []
    sentence_no = []
    for word_prob in sentence_with_importance:
        if cnt < no_of_sentences:
            sentence_no.append(word_prob[0])
            cnt = cnt+1
        else:
            break
    sentence_no.sort()
    cnt = 1
    for sentence in tokenized_sentence:
        if cnt in sentence_no:
            summary.append(sentence)
        cnt = cnt+1
    summary = "\n".join(summary)
    print("\n")
    print("Summary:")
    print("pred: ",len(summary.split(" ")))
    print("\n")
    print("act: ",len(row['Summary'].split(" ")))
    
#     break
    print(summary)
    para = para + "\n" + summary
    
outF = open('summary.txt',"w")
outF.write(para)

5


Summary:
pred:  108


act:  120
But the Cesar organisers modified their rules to allow the film to compete.
The film, directed by Jean-Pierre Jeunet, received best actress, picture and director nominations.Last November a court judged the film was too American to compete in French film festivals.
The ruling meant the movie - which was filmed in France and used French actors and technicians - was not eligible to compete for French prizes.
But Alain Terzian, president of Cesar organisers, the Academie des Arts et Techniques du Cinema, said the changes in eligibility rules, which allow films "of French expression", were made three months prior to the court decision.
The awards ceremony will be held on 26 February.
14


Summary:
pred:  290


act:  145
If you have finished Doom 3, Half Life 2 and Halo 2, don't worry.
There's a host of gaming gems set for release in 2005.WORLD OF WARCRAFTThe US reception to this game from developers Blizzard has been hugely enthusiastic, with the title t



Summary:
pred:  256


act:  187
A mobile phone chip which combines a modem and a computer processor on one bit of silicon instead of two could make phones cheaper and more powerful.The specially-designed chip, developed by Texas Instruments, could drive down the cost of making mobiles capable of 3D gaming and 30-frame-a-second video.
The technology, OMAP-Vox, is being tested by firms in Europe and Asia and could appear by the end of the year.
"We're going to drive them down into meat-and-potatoes phones that have the largest market share," said Doug Rasor, a marketing vice president at Texas.
The chip also uses much less power than conventional chips, said Texas, which means less strain on mobile battery life.More than 50 million people own a mobile in the UK, but mobile operators are keen to encourage people to move onto more sophisticated handsets that can do more.
But it faces stiff competition from the likes of Intel which is also looking to provide better chips for high-end mobi



Summary:
pred:  185


act:  176
"It wasn't a good strike," he added.
"I felt as soon as it hit my boot it had missed.
Andy Robinson said he would "keep working on the kicking" with his squad.
"We went out to play and played some very good rugby and what have France done?"
he said.
The lads showed a lot of ambition in the first half, they went out to sustain it in the second but couldn't build on it.
"We took the ball into contact, and you know when you do that it is a lottery whether the referee is going to give the penalty to your side or the other side.
There is a fine line between winning and losing, and for the second week we've been on the wrong side of that line and it hurts.
England skipper Jason Robinson admitted his side failed to cope with France's improved second-half display.
"We controlled the game in the first half but we knew that they would come out and try everything after half-time," he said.
"We made a lot of mistakes in the second half and they punished us.
Last w



Summary:
pred:  143


act:  142
Green groups say putting pressure on supply chains is a major way companies can reduce their environmental impact.
The survey is published by corporate social responsibility firm Context and branding firm SalterBaxter.Blake Lee-Harwood, campaigns director at Greenpeace in the UK, said: "It's fairly meaningless to talk about your company's direction in terms of sustainability without having detailed knowledge of your supply chain.
"It's also important to get some kind of independent assessment of your reporting.
To date there are no set standards for non-financial reporting, although the Global Reporting Initiative, an independent pro-sustainability institution, is planning to establish some.
The regulation is not expected until 2005 and the first reports under this scheme will not be published before 2006.
The US seems to lag Europe in producing corporate social responsibility reports.
The majority of European top 50 companies (44) publish them and onl



Summary:
pred:  146


act:  152
Standard Life is the latest shareholder in Deutsche Boerse to express concern at the German stock market operator's plans to buy the London Stock Exchange.It said Deutsche Boerse had to show why its planned Â£1.35bn ($2.5bn) offer for the LSE was good for shareholder value.
Reports say Standard Life, which owns a 1% stake in Deutsche Boerse, may seek a shareholder vote on the issue.
"The onus is on Deutsche Boerse's management to demonstrate why the purchase of the LSE creates more value for shareholders than other strategies, such as a buyback," said Richard Moffat, investment director of UK Equities at Standard Life Investments.
And TCI which holds about 5%, has made a request for an extraordinary shareholders meeting to be held to vote on replacing the company's entire supervisory board.
However, the LSE said the cash offer "undervalued" both its own business and the benefits of such a tie-up.
9


Summary:
pred:  254


act:  244
A senior barrister w

...



Summary:
pred:  141


act:  121
Argentina is set to close its $102.6bn (Â£53.51bn) debt restructuring offer for bondholders later on Friday, with the government hopeful that most creditors will accept the deal.The estimated loss to bondholders is up to 70% of the original value of the bonds, yet the majority are expected to accept the government's offer.
Argentina defaulted on its debt three years ago, the biggest sovereign default in modern history.
But he added that his government was close to achieving: "The best debt renegotiation in history."
The country has been in default on the $102.6bn - based on an original debt of $81.8bn plus interest - for the past three years.
By 18 February, creditors holding $41bn - or 40% of the total debt - had accepted the offer.
Sorting out its debt would enhance the country's credibility on international markets and enable it to attract more foreign investment.
5


Summary:
pred:  116


act:  87
"I think we have got to try to get a bit of respect



Summary:
pred:  207


act:  190
She praised Celtic and Rangers for their work in tackling the problem.
However, the minister said stopping sectarian abuse associated with Old Firm matches is a key objective.
Ms Jamieson stressed that sectarianism has not been confined to football but it can act as a "trigger" for tensions and violence.
Clubs have taken action in the past to ban troublesome fans and supporters' groups expressed their desire to ensure that the game is no longer tainted by the problem.Ms Jamieson said the executive should have a role in tackling the soccer troublemakers.
She said: "We can't get away from the fact that in some instances some of the religious hatred that some people try to associate with football boils over into violence.
"That is the kind of thing we want to stop and that's the kind of thing supporters' groups are very clear they don't want to be part of either, and they will work with us to try and deal with that.
"That's the kind of thing that will hit



Summary:
pred:  326


act:  258
The battle for control of Manchester United has taken another turn after the club confirmed it had received a fresh takeover approach from US business tycoon Malcolm Glazer.No formal offer has been made yet, but Manchester United have confirmed they have received a "detailed proposal" from the US entrepreneur which could lead to a bid.
Reports have put the offer at 300p per share, which would value Manchester United at about Â£800m ($1.5bn).
However, the BBC has learnt that the club is unlikely to reject the latest plan out of hand.Mr Glazer's previous offer involved borrowing large amounts of money to finance any takeover.
However, Mr Glazer's latest offer is reported to have cut the amount of borrowing needed by Â£200m.While United's board may be casting a serious eye over Mr Glazer's latest proposals, supporters remain fiercely opposed to any deal.Supporters' group Shareholders United - which has proved adept in rallying opposition to Mr Glazer's ca



Summary:
pred:  131


act:  108
US comedian Bill Cosby will not face charges stemming from an allegation of sexual misconduct.Authorities in Philadelphia said they found insufficient evidence to support the woman's allegations regarding an alleged incident in January 2004.
"He looks forward to moving on with his life," he said.
District Attorney Bruce L Castor Jr, who was in charge of the case, said that detectives could find no instance "where anyone complained to law enforcement of conduct which would constitute a criminal offence.He also said that the fact the woman waited a year before coming forward, and she had had further contact with Cosby during that time, were also factors in his decision.
"I think that's the only avenue open to her.
She felt, as we did, that it's a very strong case and she was telling the truth."
19


Summary:
pred:  352


act:  390
"It was a close race," admitted Gardener.
"I stumbled out the blocks but my experience told through.
"I still feel there's mo

95037

In [14]:
X ='No-one has come to me and said I would like to buy Nicolas Anelka.The club still owe French club PSG Ã‚Â£5m from the purchase of Anelka in May 2002.Manchester City striker Nicolas Anelka has issued an apology for criticising the ambitions of the club.But Wardle added: Theres been no bids for Nicolas Anelka.Anelka was quoted in a French newspaper as saying he would like to play in the Champions League for a bigger club.Nicolas told me that he agrees with me that we are a big club.""We are a big club.He is the heart and soul of this club and has his heart and sole in this club, and he would be very upset if I put him in the shop window.Wardle did stress that the club was not inviting any offers for England winger Shaun Wright-Phillips.Wardle was speaking at the clubs annual general meeting, where he also confirmed the club had not received any bids for the former Arsenal and Real Madrid striker.'
Y ='Manchester City striker Nicolas Anelka has issued an apology for criticising the ambitions of the club.Anelka was quoted in a French newspaper as saying he would like to play in the Champions League for a bigger club. But chairman John Wardle said: "Ive spoken to Nicolas and hes apologised for anything that might have been mistakenly taken from the French press. "We are a big club. Nicolas told me that he agrees with me that we are a big club." Wardle was speaking at the clubs annual general meeting, where he also confirmed the club had not received any bids for the former Arsenal and Real Madrid striker. The club still owe French club PSG Ã‚Â£5m from the purchase of Anelka in May 2002. He has been linked with a move to Barcelona and Liverpool, and Reds skipper Steven Gerrard also revealed he is an admirer from his time on loan at Anfield. But Wardle added: "Theres been no bids for Nicolas Anelka. No-one has come to me and said I would like to buy Nicolas Anelka."If a bid comes in for Nicolas Anelka I will speak to the board and then speak to Kevin Keegan. "If there was a bid and it was a bid of substance and worth taking then between us wed decide. "We still owe some money on Nicolas which we have clear out, so it would have to be above that." Wardle did stress that the club was not inviting any offers for England winger Shaun Wright-Phillips. He added: "Ive no intention of selling Shaun Wright-Phillips. "If someone comes with a silly bid Ill have to discuss it. "But were not putting him on the shelf to sell. He is the heart and soul of this club and has his heart and sole in this club, and he would be very upset if I put him in the shop window. "He was an academy kid here, hes just signed a new four-year deal, I dont think hed do that unless he wanted to play for Manchester City Football Club." City recently announced debts of Ã‚Â£62m, but Wardle confirmed they would try and find funds to bring in players in the January transfer window. He said: "Like Kevin Id like to see some players come in. Weve got to see what we can do - whether its a on a Bosman or not. "We will try to be creative to generate some funds. But maybe we have to start looking at clubs like Everton and Bolton to see how they have been dealing in the transfer market and do a similar type of thing.'

In [9]:
X = 'But chairman John Wardle said: "Ive spoken to Nicolas and hes apologised for anything that might have been mistakenly taken from the French press.He has been linked with a move to Barcelona and Liverpool, and Reds skipper Steven Gerrard also revealed he is an admirer from his time on loan at Anfield.No-one has come to me and said I would like to buy Nicolas Anelka."If a bid comes in for Nicolas Anelka I will speak to the board and then speak to Kevin Keegan."If there was a bid and it was a bid of substance and worth taking then between us wed decide.He added: "Ive no intention of selling Shaun Wright-Phillips."But were not putting him on the shelf to sell.He is the heart and soul of this club and has his heart and sole in this club, and he would be very upset if I put him in the shop window."He was an academy kid here, hes just signed a new four-year deal, I dont think hed do that unless he wanted to play for Manchester City Football Club."City recently announced debts of Â£62m, but Wardle confirmed they would try and find funds to bring in players in the January transfer window.But maybe we have to start looking at clubs like Everton and Bolton to see how they have been dealing in the transfer market and do a similar type of thing."'
Y ='Manchester City striker Nicolas Anelka has issued an apology for criticising the ambitions of the club.Anelka was quoted in a French newspaper as saying he would like to play in the Champions League for a bigger club. But chairman John Wardle said: "Ive spoken to Nicolas and hes apologised for anything that might have been mistakenly taken from the French press. "We are a big club. Nicolas told me that he agrees with me that we are a big club." Wardle was speaking at the clubs annual general meeting, where he also confirmed the club had not received any bids for the former Arsenal and Real Madrid striker. The club still owe French club PSG Ã‚Â£5m from the purchase of Anelka in May 2002. He has been linked with a move to Barcelona and Liverpool, and Reds skipper Steven Gerrard also revealed he is an admirer from his time on loan at Anfield. But Wardle added: "Theres been no bids for Nicolas Anelka. No-one has come to me and said I would like to buy Nicolas Anelka."If a bid comes in for Nicolas Anelka I will speak to the board and then speak to Kevin Keegan. "If there was a bid and it was a bid of substance and worth taking then between us wed decide. "We still owe some money on Nicolas which we have clear out, so it would have to be above that." Wardle did stress that the club was not inviting any offers for England winger Shaun Wright-Phillips. He added: "Ive no intention of selling Shaun Wright-Phillips. "If someone comes with a silly bid Ill have to discuss it. "But were not putting him on the shelf to sell. He is the heart and soul of this club and has his heart and sole in this club, and he would be very upset if I put him in the shop window. "He was an academy kid here, hes just signed a new four-year deal, I dont think hed do that unless he wanted to play for Manchester City Football Club." City recently announced debts of Ã‚Â£62m, but Wardle confirmed they would try and find funds to bring in players in the January transfer window. He said: "Like Kevin Id like to see some players come in. Weve got to see what we can do - whether its a on a Bosman or not. "We will try to be creative to generate some funds. But maybe we have to start looking at clubs like Everton and Bolton to see how they have been dealing in the transfer market and do a similar type of thing.'


In [10]:
# tokenization 
X_list = word_tokenize(X)  
Y_list = word_tokenize(Y) 
  
# sw contains the list of stopwords 
sw = stopwords.words('english')  
l1 =[];l2 =[] 
  
# remove stop words from the string 
X_set = {w for w in X_list if not w in sw}  
Y_set = {w for w in Y_list if not w in sw} 
  
# form a set containing keywords of both strings  
rvector = X_set.union(Y_set)  
for w in rvector: 
    if w in X_set: l1.append(1) # create a vector 
    else: l1.append(0) 
    if w in Y_set: l2.append(1) 
    else: l2.append(0) 
c = 0
  
# cosine formula  
for i in range(len(rvector)): 
        c+= l1[i]*l2[i] 
cosine = c / float((sum(l1)*sum(l2))**0.5) 
print("similarity: ", cosine) 

similarity:  0.7750658792607136


In [11]:
!pip install rouge

In [12]:
from rouge import Rouge 

hypothesis = "the #### transcript is a written version of each day 's cnn student news program use this transcript to he    lp students with reading comprehension and vocabulary use the weekly newsquiz to test your knowledge of storie s you     saw on cnn student news"

reference = "this page includes the show transcript use the transcript to help students with reading comprehension and     vocabulary at the bottom of the page , comment for a chance to be mentioned on cnn student news . you must be a teac    her or a student age # # or older to request a mention on the cnn student news roll call . the weekly newsquiz tests     students ' knowledge of even ts in the news"

rouge = Rouge()
scores = rouge.get_scores(X, Y)

In [13]:
scores

[{'rouge-1': {'f': 0.7026239021305325,
   'p': 0.9917695473251029,
   'r': 0.5440180586907449},
  'rouge-2': {'f': 0.6783625685268972,
   'p': 0.9586776859504132,
   'r': 0.5248868778280543},
  'rouge-l': {'f': 0.808510633461408,
   'p': 0.987012987012987,
   'r': 0.6846846846846847}}]

In [15]:
rouge = Rouge()
scores = rouge.get_scores(X, Y)
scores

[{'rouge-1': {'f': 0.555194801155391,
   'p': 0.9884393063583815,
   'r': 0.3860045146726862},
  'rouge-2': {'f': 0.527687292383792,
   'p': 0.9418604651162791,
   'r': 0.3665158371040724},
  'rouge-l': {'f': 0.6215384572088049,
   'p': 0.9805825242718447,
   'r': 0.45495495495495497}}]